# Differential Privacy for MNIST Dataset
*Project Background:* <br>
We, as a student, has a labeled private dataset and an unlabeled public dataset. We would like to collaberate with certain number of teachers to label our public dataset by training their similar datasets. In order to protect the data privacy of the teachers, we agree to add global noise to the labels they return for the public dataset. Then we label our public dataset based on the labels that most teachers agree. We combine the new labels with the public dataset and train with our private dataset. <br>

*PATE analysis* is performed on teachers' predictions and the new labels with certain level of epsilon indicating how much information leakage is allowed.

In [0]:
# install syft package to use Private Aggregation of Teacher Ensembles (PATE)
# !pip install syft

In [0]:
# import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, TensorDataset
from torch.utils.data.dataset import random_split
from torch import nn, optim
import torch.nn.functional as F
from syft.frameworks.torch.differential_privacy import pate
import matplotlib.pyplot as plt

### Section 1: Download and load MNIST dataset

*   **Trainset**: divide among teachers without overlapping
*   **Testset**: split into private(labeled) and public(unlabeled) datasets; use models trained by teachers to label public dataset and train with the private dataset



In [0]:
# tranform for data normalization
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, ), (0.5, )),    # MNIST data is 2D
                                ])

# download training and test data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

In [0]:
# training and test data info
print('num training data: {0}'.format(len(trainset)))
print('num test data: {0}'.format(len(testset)))

num training data: 60000
num test data: 10000


### Section 2: Functions


*   `teacher_data_allocator`: evenly allocate training dataset to the number of teachers as their private datasets
*   `private_public_allocator`: split test dataset as the student's private dataset and public dataset
*   `train`: model training with training and valid datasets
*   `get_label_testset`: get labels of the student's public dataset from all teachers
*   `add_global_noise`: add global noise to the labels that most teachers agree
*   `create_labeled_public_db`: extract images from the public dataset and combine these images with the new labels; create a new dataloader for this new dataset



In [0]:
# divide unique training datasets among teachers
# split each dataset into training and valid sets
def teacher_data_allocator(trainset, batch_size=64, num_teachers=10, valid_train_split=0.3):
  data_per_teacher = len(trainset) // num_teachers    # round to the nearest integer
  train_loaders = []
  valid_loaders = []
  for i in range(num_teachers):
    if i == num_teachers - 1:
      ind = list(range(i * data_per_teacher, len(trainset)))
    else:
      ind = list(range(i * data_per_teacher, (i + 1) * data_per_teacher))
    data = Subset(trainset, ind)
    split = int(valid_train_split * len(data))
    valid_data, train_data = random_split(data, [split, len(data) - split])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)
    train_loaders.append(train_loader)
    valid_loaders.append(valid_loader)
  return train_loaders, valid_loaders

In [0]:
# divide testset into private and public
def private_public_allocator(dataset, batch_size=64, split=0.7):
  split_ind = int(len(dataset) * split)
  
  # validset and testset
  # validset is smaller because dataset of each teacher is small
  private_ind = list(range(0, split_ind))
  public_ind = list(range(split_ind, len(dataset)))
  private_set = Subset(dataset, private_ind)
  public_set = Subset(dataset, public_ind)

  # validloader and testloader
  private_loader = torch.utils.data.DataLoader(private_set, batch_size=batch_size, shuffle=True)
  public_loader = torch.utils.data.DataLoader(public_set, batch_size=batch_size, shuffle=False)
  return private_loader, public_loader

In [0]:
# method for training
def train(model, criterion, optimizer, trainloader, validloader, epochs=10):
  model = model
  criterion = criterion
  optimizer = optimizer
  epochs = epochs

  train_losses, valid_losses = [], []
  for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:

      optimizer.zero_grad()

      log_ps = model(inputs)
      loss = criterion(log_ps, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
  
    else:
      valid_loss = 0
      acc = 0

      # turn off gradients for validation, saving memory and computations
      with torch.no_grad():
        model.eval()
        for inputs, labels in validloader:

          log_ps = model(inputs)
          valid_loss += criterion(log_ps, labels)

          ps = torch.exp(log_ps)
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          acc += torch.mean(equals.type(torch.FloatTensor))

      model.train()

      train_losses.append(running_loss / len(trainloader))
      valid_losses.append(valid_loss / len(validloader))

      print('Epoch: {}/{}.. '.format(e+1, epochs),
            'Training Loss: {:.3f}.. '.format(running_loss / len(trainloader)),
            'Valid Loss: {:.3f}.. '.format(valid_loss / len(validloader)),
            'Valid Accuracy: {:.3f} '.format(acc / len(validloader)),
            '')
  return model

In [0]:
# label testset by models
def get_label_testset(models, testloader):
  test_labels = []
  for model in models:
    test_label = []
    for inputs, _ in testloader:
      with torch.no_grad():
        log_ps = model(inputs)
        ps = torch.exp(log_ps)
      top_p, top_class = ps.topk(1, dim=1)
      test_label.append(top_class.squeeze().tolist())
    test_label = sum(test_label, [])
    test_labels.append(test_label)
  return test_labels

In [0]:
# get predictions with global noise
def add_global_noise(preds, epsilon=0.1):
  labels_with_noise = []
  for pred in preds:    # go thru entries
    label_counts = np.bincount(pred, minlength=preds.shape[1])

    epsilon = epsilon
    beta = 1 / epsilon

    for i in range(len(label_counts)):
      label_counts[i] += np.random.laplace(0, beta, 1)
  
    new_label = np.argmax(label_counts)
    labels_with_noise.append(new_label)
  return np.array(labels_with_noise)

In [0]:
# create a new dataloader for the public dataset
# using the new labels added global noise
def create_labeled_public_db(dataloader, labels, batch_size=64):
  image_list = []
  for image, _ in dataloader:
    image_list.append(image)
  data = np.vstack(image_list)
  new_dataloader = list(zip(data, labels))
  new_dataloader = torch.utils.data.DataLoader(new_dataloader, shuffle=False, batch_size=batch_size)
  return new_dataloader

### Section 3: Define neural network model structure

In [0]:
# Classifier for model creation
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 10)

    # dropout module with 0.2 drop probability
    self.dropout = nn.Dropout(p=0.2)
  
  def forward(self, x):
    # flatten input tensor
    x = x.view(x.shape[0], -1)

    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))

    x = F.log_softmax(self.fc4(x), dim=1)

    return x

### Section 4: Model training and PATE analysis
We first ask the teachers to train their data and use these models to label the student's public data. After adding the global noise to create the new labels for the public dataset, the student then trains his/her private (training set) and public (valid set) data. PATE analysis returns independent and dependent epsilons show how much information has been leaked and how much teachers agree with each other.

In [0]:
# allocate training dataset to teachers
# split each of these datasets into training and valid datasets
teachers_train, teachers_valid = teacher_data_allocator(trainset)
print(len(teachers_train))
print(len(teachers_valid))

10
10


In [0]:
# allocate test dataset to the student
# split the dataset into private and public datasets
private_loader, public_loader = private_public_allocator(testset)
print(len(private_loader))
print(len(public_loader))

110
47


In [0]:
# get models trained by each teacher
teacher_models = []
for teacher_train, teacher_valid in zip(teachers_train, teachers_valid):
  model = Classifier()
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.003)
  teacher_model = train(model, criterion, optimizer, teacher_train, teacher_valid)
  teacher_models.append(teacher_model)

Epoch: 1/10..  Training Loss: 1.349..  Valid Loss: 0.518..  Valid Accuracy: 0.847  
Epoch: 2/10..  Training Loss: 0.592..  Valid Loss: 0.354..  Valid Accuracy: 0.894  
Epoch: 3/10..  Training Loss: 0.447..  Valid Loss: 0.345..  Valid Accuracy: 0.894  
Epoch: 4/10..  Training Loss: 0.383..  Valid Loss: 0.315..  Valid Accuracy: 0.907  
Epoch: 5/10..  Training Loss: 0.347..  Valid Loss: 0.307..  Valid Accuracy: 0.905  
Epoch: 6/10..  Training Loss: 0.323..  Valid Loss: 0.283..  Valid Accuracy: 0.917  
Epoch: 7/10..  Training Loss: 0.289..  Valid Loss: 0.269..  Valid Accuracy: 0.924  
Epoch: 8/10..  Training Loss: 0.290..  Valid Loss: 0.228..  Valid Accuracy: 0.934  
Epoch: 9/10..  Training Loss: 0.248..  Valid Loss: 0.221..  Valid Accuracy: 0.939  
Epoch: 10/10..  Training Loss: 0.261..  Valid Loss: 0.229..  Valid Accuracy: 0.929  
Epoch: 1/10..  Training Loss: 1.423..  Valid Loss: 0.634..  Valid Accuracy: 0.805  
Epoch: 2/10..  Training Loss: 0.655..  Valid Loss: 0.415..  Valid Accuracy:

In [0]:
# predict labels for testset from each teacher
# row = num of test data
# col = predictions from each teacher
preds = get_label_testset(teacher_models, public_loader)
preds = np.array([np.array(p) for p in preds]).transpose(1, 0)

In [0]:
preds_with_noise = add_global_noise(preds, epsilon=0.6)
len(preds_with_noise)

3000

In [0]:
# PATE analysis
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds.T, indices=preds_with_noise, noise_eps=0.1, delta=1e-5)
print('Data dependent epsilon:', data_dep_eps)
print('Data independent epsilon:', data_ind_eps)

Data dependent epsilon: 131.5129254649778
Data independent epsilon: 131.51292546497027


In [0]:
# create new dataloader for the public data
# using the original function
public_loader_labeled = create_labeled_public_db(public_loader, preds_with_noise)
len(public_loader_labeled)

47

In [0]:
# train model using private and public datasets
my_model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(my_model.parameters(), lr=1e-3)
train(model=my_model, criterion=criterion, optimizer=optimizer, trainloader=private_loader, validloader=public_loader_labeled)

Epoch: 1/10..  Training Loss: 1.247..  Valid Loss: 0.735..  Valid Accuracy: 0.810  
Epoch: 2/10..  Training Loss: 0.558..  Valid Loss: 0.612..  Valid Accuracy: 0.859  
Epoch: 3/10..  Training Loss: 0.420..  Valid Loss: 0.575..  Valid Accuracy: 0.884  
Epoch: 4/10..  Training Loss: 0.349..  Valid Loss: 0.578..  Valid Accuracy: 0.894  
Epoch: 5/10..  Training Loss: 0.310..  Valid Loss: 0.538..  Valid Accuracy: 0.909  
Epoch: 6/10..  Training Loss: 0.264..  Valid Loss: 0.574..  Valid Accuracy: 0.902  
Epoch: 7/10..  Training Loss: 0.254..  Valid Loss: 0.601..  Valid Accuracy: 0.910  
Epoch: 8/10..  Training Loss: 0.230..  Valid Loss: 0.630..  Valid Accuracy: 0.904  
Epoch: 9/10..  Training Loss: 0.212..  Valid Loss: 0.614..  Valid Accuracy: 0.905  
Epoch: 10/10..  Training Loss: 0.186..  Valid Loss: 0.620..  Valid Accuracy: 0.912  


Classifier(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)